In [80]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2 as cv
import mediapipe as mp

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

### Creating a model based on hands tracking algorithm

In [4]:
hands_tracking = np.load('Other signs/hands-tracking-data.npy')

In [5]:
labels = np.load(r'Other signs\training-labels.npy')

In [16]:
test = np.load('Other signs/hands-tracking-test.npy')
test_label = np.load(r'Other signs\testing-labels.npy')

In [6]:
labels.shape

(400,)

In [89]:
type(hands_tracking)

numpy.ndarray

In [7]:
hands_tracking = np.asarray(hands_tracking).astype('float32').reshape(400,250,250,1)

In [8]:
hands_tracking /= 255.0

In [9]:
hands_tracking.shape

(400, 250, 250, 1)

In [12]:
model = Sequential()
model.add(layers.Conv2D(32,(3,3),activation='relu',input_shape=(250,250,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(32,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(500,activation='relu'))
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(5,activation='softmax'))

In [13]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 248, 248, 32)      320       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 124, 124, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 122, 122, 32)      9248      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 61, 61, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 59, 59, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 29, 29, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 27, 27, 64)       

In [14]:
model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),metrics=['accuracy'])

In [15]:
history = model.fit(hands_tracking,labels,epochs=10)

Epoch 1/10
13/13 [==============================] - 22s 1s/step - loss: 1.6138 - accuracy: 0.2487
Epoch 2/10
13/13 [==============================] - 17s 1s/step - loss: 1.3568 - accuracy: 0.4192
Epoch 3/10
13/13 [==============================] - 17s 1s/step - loss: 1.0262 - accuracy: 0.6116
Epoch 4/10
13/13 [==============================] - 17s 1s/step - loss: 0.6716 - accuracy: 0.7944
Epoch 5/10
13/13 [==============================] - 17s 1s/step - loss: 0.4846 - accuracy: 0.8180
Epoch 6/10
13/13 [==============================] - 18s 1s/step - loss: 0.3320 - accuracy: 0.8822
Epoch 7/10
13/13 [==============================] - 24s 2s/step - loss: 0.2471 - accuracy: 0.9067
Epoch 8/10
13/13 [==============================] - 21s 2s/step - loss: 0.2023 - accuracy: 0.9067
Epoch 9/10
13/13 [==============================] - 21s 2s/step - loss: 0.1925 - accuracy: 0.9229
Epoch 10/10
13/13 [==============================] - 21s 2s/step - loss: 0.2082 - accuracy: 0.9193


In [17]:
test = np.asarray(test).astype('float32').reshape(146,250,250,1)
test /= 255.0

In [18]:
model.evaluate(test,test_label)

5/5 [==============================] - 3s 358ms/step - loss: 1.2336 - accuracy: 0.7603


[1.233563780784607, 0.7602739930152893]

In [85]:
model.save("models/Hands-tracking-detection")

INFO:tensorflow:Assets written to: models/Hands-tracking-detection\assets


### Using the model to do further implementaion

In [87]:
model = tf.keras.models.load_model('models/Hands-tracking-detection')

In [57]:
WIDTH = 1366
HEIGHT = 768
x1 = WIDTH//2 - 300
x2 = WIDTH//2 + 300
y1 = HEIGHT//2 - 300
y2 = HEIGHT//2 + 300

In [62]:
num2str = {0:'Call me',1:'Good job',2:'Good luck',3:'Ok',4:'Victory'}

In [59]:
class handTracker():
    def __init__(self, mode=False, maxHands=2, detectionCon=0.5,modelComplexity=1,trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.modelComplex = modelComplexity
        self.trackCon = trackCon
        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands,self.modelComplex,
                                        self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils

    def handsFinder(self,image,draw=True,bgr=False):
        if bgr:
            imageRGB = cv.cvtColor(image,cv.COLOR_BGR2RGB)
            self.results = self.hands.process(imageRGB)
        else:
            self.results = self.hands.process(image)
        
        black_screen = np.zeros((250, 250, 3), dtype = "uint8")
        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(black_screen, handLms, self.mpHands.HAND_CONNECTIONS)
                    self.mpDraw.draw_landmarks(image, handLms, self.mpHands.HAND_CONNECTIONS)
        return image, cv.cvtColor(cv.resize(black_screen,(250,250),interpolation=cv.INTER_AREA),cv.COLOR_RGB2GRAY)

In [98]:
cap = cv.VideoCapture(0)
tracker = handTracker()
while True:
    success,frame = cap.read()
    image, black = tracker.handsFinder(frame[y1:y2,x1:x2],bgr=True)
    cv.rectangle(image, (x1,y2),(x2,y2),(0,255,0), 1)
    black.resize((1,250,250,1))
    prob = model.predict(black)
    op = np.argmax(prob)
    cv.putText(image, f'{num2str[op]} \n {prob[0,op]}',org=(10,50),fontFace=cv.FONT_HERSHEY_SIMPLEX,color=(0, 255, 0),fontScale=1.0,thickness=2)

    cv.imshow("Video",image)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
    
cap.release()
cv.destroyAllWindows()